In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
from ensemble_processing import load_data, load, save
import keras

/Users/neilkloot/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500

In [3]:
def return_week_median(df):
    # assumes a datetime index
    return df.resample('1W').median()


# test_df = pd.DataFrame({
#     'dates': ['2018-01-02', '2018-01-03', '2018-01-04', '2018-03-05', '2019-04-09'],
#     'val': [1, 2, 2, 3, 3],
#     'date2': ['2019-01-02', '2019-01-03','2019-02-02', '2019-02-03', '2019-05-05']
# })
# test_df['dates'] = pd.to_datetime(test_df['dates'], errors='coerce')
# test_df['date2'] = pd.to_datetime(test_df['date2'], errors='coerce')
# test_df.sort_values(by=['dates'], inplace=True)
# print(test_df.dtypes)
# test_df.set_index('dates', inplace=True)
# print(test_df)
# print(test_df.index)
# print(test_df.index.week, test_df.index.year)
# return_week_median(test_df)

In [4]:
import sklearn
sklearn.__version__

'0.20.3'

In [5]:
%load_ext autoreload
%autoreload 2

In [23]:
# Load data
print('Loading data...')
all_df = pd.read_pickle('../data/ml-20190115-processed.pkl.gz', compression='gzip')


Loading data...


## Create one vector with all symbols per week

In [24]:
import math


# experiment_symbols = ['WAX', 'IVV', 'WESN', 'IAA', 'XRO', 'MTB', 'MXR', 'RCP', 'ISX', 'IMI']
# experiment_symbols = ['GSW','CUA','FLT','OVN','FDM','SVW','FPH','JHC','TRS','SGP','DOW','AOG',
#                       'HSN','SWM','CMI','BCI','SWJ','MYX','CZA','VMT','CLQ','CSE','OOK','BRL',
#                       'ENC','SIO','ECG','MHC','ICT','NTM','REV','FND','GOE','QMN','DSB','KRS',
#                       'LAU','GSZ','SAS','OCC','INK','EVM','XST','WWI','BSM','TKL','ZMI','DGO',
#                       'ALT','RNO','IRC','AYM','IVG','MPE','OAR','DSE','KTL']

experiment_symbols = ['GSW','VGAD','CUA','TIX','FLT','PTN','OVN','IRU','FDM','HDF','SVW','BWX',
                      'FPH','AGL','JHC','HIN','TRS','HGO','SGP','AVN','DOW','NVL','AOG','BGL',
                      'HSN','CSS','SWM','RIC','CMI','MKE','BCI','IBG','SWJ','IXP','MYX','RNY',
                      'CZA','SRV','VMT','CDM','CLQ','BPS','CSE','ORR','OOK','EOS','BRL','BWP',
                      'ENC','FCT','SIO','SZG','ECG','SXA','MHC','PHK','ICT','CLH','NTM','TAS',
                      'REV','CII','FND','ZGM','GOE','SSI','QMN','EAL','DSB','PNX','KRS','RXH',
                      'LAU','CNW','GSZ','IBC','SAS','EAS','OCC','AIS','INK','AIK','EVM','MSV',
                      'XST','GMR','WWI','JYC','BSM','VRX','TKL','WFE','ZMI','SHK','DGO','BD1',
                      'ALT','SES','RNO','MXC','IRC','GTR','AYM','RLC','IVG','MDG','MPE','MOT',
                      'OAR','HGL','DSE','VII','KTL']

WHOLE_MARKET_COLUMNS = ['quoteDate', 'allordpreviousclose', 'allorddayshigh', 'allorddayslow', 'asxpreviousclose',
                        'asxdayshigh', 'asxdayslow', '640106_A3597525W', 'FIRMMCRT', 'FXRUSD', 'GRCPAIAD',
                        'GRCPAISAD', 'GRCPBCAD', 'GRCPBCSAD', 'GRCPBMAD', 'GRCPNRAD', 'GRCPRCAD', 'H01_GGDPCVGDP',
                        'H01_GGDPCVGDPFY', 'H05_GLFSEPTPOP']

SYMBOL_COLS = ['quoteDate', 'adjustedPrice', 'AINTCOV', 'Beta', 'BookValuePerShareYear', 'CashPerShareYear', 
               'DPSRecentYear', 'EPS', 'Float', 'MarketCap', 'OperatingMargin', 'PE', 'ReturnOnEquityYear', 
               'TotalDebtToEquityYear', 'exDividendDate']
               

In [32]:
### 2D vector - flat

#### Get unique whole market vals (shared across all symbols)
print('Total data loaded', len(all_df), 'records')
daily_symbols_df = all_df[all_df['symbol'].isin(experiment_symbols)]

print('Records being used', len(daily_symbols_df))

# Determine the min and max range for our symbols
all_possible_dates = daily_symbols_df['quoteDate'].dropna().drop_duplicates()
overall_min_date = all_possible_dates.min()
overall_max_date = all_possible_dates.max()
print('Min date for recordset', overall_min_date)
print('Max date for recordset', overall_max_date)

reference_whole_market_data = daily_symbols_df[WHOLE_MARKET_COLUMNS].drop_duplicates()

# Ensure there is only one record per day
reference_whole_market_data = reference_whole_market_data.groupby(
    'quoteDate').first()
reference_whole_market_data['quoteDate'] = reference_whole_market_data.index

# Convert records to weekly medi    
median_whole_market_data = return_week_median(reference_whole_market_data)
print('Number of weeks in dataset', len(median_whole_market_data))


# Retrieve date parameters for start and finish
symbols_median_dfs = []

# Create symbol dfs with daily data, the convert to weekly median data
for symbol in experiment_symbols:
    # Filter to model data for this symbol and re-set the pandas indexes
    model_data = daily_symbols_df.loc[daily_symbols_df['symbol'] == symbol][SYMBOL_COLS]
    
    print('Symbol:', symbol, 'number of daily records:', len(model_data))
        
#     print('Converting to week median values')    
    # Conert date to number
    model_data['exDividendRelative'] = model_data['exDividendDate'] - model_data['quoteDate']
    # convert string difference value to integer
    model_data['exDividendRelative'] = model_data['exDividendRelative'].apply(
        lambda x: np.nan if pd.isnull(x) else x.days)
    # Make sure it is the minimum data type size
    model_data.loc[:, 'exDividendRelative'] = model_data['exDividendRelative'].astype('int32', errors='ignore')
        
    median_model_data = return_week_median(model_data)
    
    # Re-add in key categorical values
    median_model_data['symbol'] = symbol
                
    # Calculate weekly change percentage allowing for 0 values
    median_model_data['price_change_perc'] = (median_model_data['adjustedPrice'] - \
                                                median_model_data['adjustedPrice'].shift(1)) / \
                                                median_model_data['adjustedPrice'].shift(1).clip(lower=0.1) * \
                                                100

    # Calculate target (Y) values for the following 8 weeks
    for week_num in range(1, 9):
        shift_amount = week_num * -1
        median_model_data['y_' + str(week_num)] = (median_model_data['adjustedPrice'] - \
                                                median_model_data['adjustedPrice'].shift(shift_amount)) / \
                                                median_model_data['adjustedPrice'].clip(lower=0.1) * \
                                                100
            
    print('Symbol:', symbol, 'number of median records:', len(median_model_data), 'average change:', np.mean(median_model_data['price_change_perc']))
    print('-'*10)
                
    symbols_median_dfs.append(median_model_data)
    

# Create concatenated dataframe with all weekly median data
print('Creating concatenated dataframes')
symbols_median_df = pd.concat(symbols_median_dfs)

print('Number of symbol median records', len(symbols_median_df))
# symbols_median_df

target_cols = []
target_col_names = []

# Work through median values for each symbol and to the whole market data or -9999 for no value
for symbol in experiment_symbols:
    symbol_df = symbols_median_df.loc[symbols_median_df['symbol'] == symbol]
#     print(symbol_df.columns)
    symbol_df.drop(['symbol'], axis=1, inplace=True)
    symbol_df = symbol_df.add_prefix(symbol + '_')
    
    # Add target column name for future y data
    target_cols.append(symbol + '_price_change_perc')
    target_col_names.append(symbol)

    median_whole_market_data = median_whole_market_data.merge(symbol_df, left_index=True, right_index=True, 
                                                              how='left', suffixes = ('wm_', '_' + symbol))

    median_whole_market_data['month'] = median_whole_market_data.index.month
    median_whole_market_data['week_num'] = median_whole_market_data.index.weekofyear
    median_whole_market_data['year'] = median_whole_market_data.index.year
    
#     median_whole_market_data[symbol_df.columns.values] = median_whole_market_data[symbol_df.columns.values].fillna(-999)
    median_whole_market_data.fillna(-999, inplace=True)

    
print(median_whole_market_data.shape)
print('Num target cols', len(target_cols))

# Construct set of cols which are the future 8 week targets
eight_week_target_cols = []
for symbol in target_col_names:
    for week_num in range(1, 9):
        eight_week_target_cols.append(symbol + '_y_' + str(week_num))
    
# Create target series and remove target cols from X data 
whole_market_targets = median_whole_market_data[eight_week_target_cols]

median_whole_market_data.drop(eight_week_target_cols, axis=1, inplace=True)


Total data loaded 5384641 records
Records being used 268358
Min date for recordset 2007-07-02 00:00:00
Max date for recordset 2019-01-14 00:00:00
Number of weeks in dataset 603
Symbol: GSW number of daily records: 593
Symbol: GSW number of median records: 111 average change: 1.5623096227645874
----------
Symbol: VGAD number of daily records: 911
Symbol: VGAD number of median records: 181 average change: 0.14820389449596405
----------
Symbol: CUA number of daily records: 794
Symbol: CUA number of median records: 164 average change: 0.22959904372692108
----------
Symbol: TIX number of daily records: 1049
Symbol: TIX number of median records: 216 average change: 0.32570168375968933
----------
Symbol: FLT number of daily records: 3009
Symbol: FLT number of median records: 603 average change: 0.39136746525764465
----------
Symbol: PTN number of daily records: 764
Symbol: PTN number of median records: 157 average change: -1.4026567935943604
----------
Symbol: OVN number of daily records: 697

Symbol: FND number of median records: 599 average change: 0.007512769661843777
----------
Symbol: ZGM number of daily records: 2364
Symbol: ZGM number of median records: 483 average change: -0.24111390113830566
----------
Symbol: GOE number of daily records: 2222
Symbol: GOE number of median records: 454 average change: -0.1262877881526947
----------
Symbol: SSI number of daily records: 789
Symbol: SSI number of median records: 161 average change: -0.36029860377311707
----------
Symbol: QMN number of daily records: 2407
Symbol: QMN number of median records: 488 average change: -0.04624941945075989
----------
Symbol: EAL number of daily records: 2400
Symbol: EAL number of median records: 491 average change: -0.2165878415107727
----------
Symbol: DSB number of daily records: 1640
Symbol: DSB number of median records: 336 average change: -0.05726109817624092
----------
Symbol: PNX number of daily records: 2850
Symbol: PNX number of median records: 570 average change: -0.12348673492670059


/Users/neilkloot/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(603, 2621)
Num target cols 113


In [31]:
median_whole_market_data.columns


Index(['allordpreviousclose', 'allorddayshigh', 'allorddayslow',
       'asxpreviousclose', 'asxdayshigh', 'asxdayslow', '640106_A3597525W',
       'FIRMMCRT', 'FXRUSD', 'GRCPAIAD',
       ...
       'KTL_exDividendRelative', 'KTL_price_change_perc', 'KTL_y_1', 'KTL_y_2',
       'KTL_y_3', 'KTL_y_4', 'KTL_y_5', 'KTL_y_6', 'KTL_y_7', 'KTL_y_8'],
      dtype='object', length=2621)

In [ ]:
### X - 3D vector - date, symbol, features
### y - 2D vector - date, symbol_1 val, symbol_2 val ... 

### 2D vector - flat

# ConvLSTM2D expects input as [samples, timesteps, rows, columns, features]
#                             [sample-> 16 input weeks -> 1 -> whole market vals + symbol vals features]

#### Get unique whole market vals (shared across all symbols)
print('Total data loaded', len(all_df), 'records')
daily_symbols_df = all_df[all_df['symbol'].isin(experiment_symbols)]

print('Records being used', len(daily_symbols_df))

# Determine the min and max range for our symbols
all_possible_dates = daily_symbols_df['quoteDate'].dropna().drop_duplicates()
overall_min_date = all_possible_dates.min()
overall_max_date = all_possible_dates.max()
print('Min date for recordset', overall_min_date)
print('Max date for recordset', overall_max_date)

reference_whole_market_data = daily_symbols_df[WHOLE_MARKET_COLUMNS].drop_duplicates()

# Ensure there is only one record per day
reference_whole_market_data = reference_whole_market_data.groupby(
    'quoteDate').first()
reference_whole_market_data['quoteDate'] = reference_whole_market_data.index

# Convert records to weekly medi    
median_whole_market_data = return_week_median(reference_whole_market_data)
print('Number of weeks in dataset', len(median_whole_market_data))


# Retrieve date parameters for start and finish
symbols_median_dfs = []

# Create symbol dfs with daily data, the convert to weekly median data
for symbol in experiment_symbols:
    # Filter to model data for this symbol and re-set the pandas indexes
    model_data = daily_symbols_df.loc[daily_symbols_df['symbol'] == symbol][SYMBOL_COLS]
    
    print('Symbol:', symbol, 'number of daily records:', len(model_data))
        
#     print('Converting to week median values')    
    # Conert date to number
    model_data['exDividendRelative'] = model_data['exDividendDate'] - model_data['quoteDate']
    # convert string difference value to integer
    model_data['exDividendRelative'] = model_data['exDividendRelative'].apply(
        lambda x: np.nan if pd.isnull(x) else x.days)
    # Make sure it is the minimum data type size
    model_data.loc[:, 'exDividendRelative'] = model_data['exDividendRelative'].astype('int32', errors='ignore')
        
    median_model_data = return_week_median(model_data)
    
    # Re-add in key categorical values
    median_model_data['symbol'] = symbol
        
    # Calculate weekly change percentage allowing for 0 values
    median_model_data['price_change_perc'] = (median_model_data['adjustedPrice'] - \
                                                median_model_data['adjustedPrice'].shift(1)) / \
                                                median_model_data['adjustedPrice'].shift(1).clip(lower=0.1) * \
                                                100

    print('Symbol:', symbol, 'number of median records:', len(median_model_data), 'average change:', np.mean(median_model_data['price_change_perc']))
    print('-'*10)
                
    symbols_median_dfs.append(median_model_data)
    symbol_counter += 1
    

# Create concatenated dataframe with all weekly median data
print('Creating concatenated dataframes')
symbols_median_df = pd.concat(symbols_median_dfs)

print('Number of symbol median records', len(symbols_median_df))
# symbols_median_df

target_cols = []

# Work through median values for each symbol and to the whole market data or -999 for no value
for symbol in experiment_symbols:
    symbol_df = symbols_median_df.loc[symbols_median_df['symbol'] == symbol]
#     print(symbol_df.columns)
    symbol_df.drop(['symbol'], axis=1, inplace=True)
    symbol_df = symbol_df.add_prefix(symbol + '_')
    
    # Add target column name for future y data
    target_cols.append(symbol)

    median_whole_market_data = median_whole_market_data.merge(symbol_df, left_index=True, right_index=True, 
                                                              how='left', suffixes = ('wm_', '_' + symbol))

#     median_whole_market_data[symbol_df.columns.values] = median_whole_market_data[symbol_df.columns.values].fillna(-999)
    median_whole_market_data.fillna(-999, inplace=True)

    
print(median_whole_market_data.shape)
print('Num target cols', len(target_cols))


    
# Create tagrte vector using the stationary vals for the following week 
whole_market_targets = median_whole_market_data[target_cols].shift(-1)

In [9]:
del all_df

In [ ]:
# Save dataframes
df_train_x.to_pickle("../data/ml-rnn-experiment-baseline.pkl.gz", compression='gzip')

# Pre-processing - separate symbol rows

In [ ]:
def remove_future_cols(df):
    cols_to_drop = []
    for col in df.columns:
        if 'future' in col or col.startswith('_'):
            cols_to_drop.append(col)

    return df.drop(cols_to_drop, axis=1)

In [ ]:
from ensemble_processing import execute_one_hot_string_encoder, train_rar_encoder
from processing_constants import COLUMNS_TO_REMOVE
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.preprocessing import OrdinalEncoder

target_col = 'price_change_perc'

all_train_x = df_train_x.dropna(subset=[target_col])
all_train_x = remove_future_cols(all_train_x)
all_train_x.drop(COLUMNS_TO_REMOVE, axis=1, errors='ignore', inplace=True)
categorical_vals = ['symbol', 'GICSSector', 'GICSIndustryGroup', 'GICSIndustry']
symbol_lookup = all_train_x['symbol']

print('Encoding categorical vals')
cat_enc = OrdinalEncoder()
all_train_x[categorical_vals] = cat_enc.fit_transform(all_train_x[categorical_vals].values)

all_val_cols = []

for col in all_train_x.columns:
    if col != target_col:
        all_val_cols.append(col)
        
print(all_val_cols)

print('Training and executing imputer')
imputer = Imputer(strategy='median')
all_train_x[all_val_cols] = imputer.fit_transform(all_train_x[all_val_cols].values)

print('Training and executing scaler')
scaler = MinMaxScaler(feature_range=(0, 1))
all_train_x[all_val_cols] = scaler.fit_transform(all_train_x[all_val_cols].values)

print('Done')

In [ ]:
# all_train_x.isnull().sum()
# all_train_x['_13']
# all_train_x.drop(['_13'], axis=1, inplace=True)
# all_df['exDividendDate'].isnull()
symbol_lookup == 'GSW'

## Preprocessing - flat all symbols in each row

In [10]:
from sklearn.preprocessing import MinMaxScaler

print('Checking for nans')
x_msk = median_whole_market_data.isnull()
y_msk = whole_market_targets.isnull()

print('X nans:', x_msk.sum(), 'y nans:', y_msk.sum())

total_ys = np.sum(y_msk.sum())
print('Total y nans:', total_ys)
print('Expected y nans:', len(target_cols))

if total_ys != len(target_cols):
    print('WARNING - UNEXPECTED NaNs')


print('Training and executing scaler')
x_scaler = MinMaxScaler(feature_range=(0, 1))
all_x = x_scaler.fit_transform(median_whole_market_data[:-1].values)


# y_scaler = MinMaxScaler(feature_range=(0, 1))
# all_train_y = y_scaler.fit_transform(whole_market_targets[:-1].values)

all_y = whole_market_targets[:-1].values

print('Scaling done')

Checking for nans
X nans: allordpreviousclose           0
allorddayshigh                0
allorddayslow                 0
asxpreviousclose              0
asxdayshigh                   0
asxdayslow                    0
640106_A3597525W              0
FIRMMCRT                      0
FXRUSD                        0
GRCPAIAD                      0
GRCPAISAD                     0
GRCPBCAD                      0
GRCPBCSAD                     0
GRCPBMAD                      0
GRCPNRAD                      0
GRCPRCAD                      0
H01_GGDPCVGDP                 0
H01_GGDPCVGDPFY               0
H05_GLFSEPTPOP                0
GSW_adjustedPrice             0
GSW_AINTCOV                   0
GSW_Beta                      0
GSW_BookValuePerShareYear     0
GSW_CashPerShareYear          0
GSW_DPSRecentYear             0
GSW_EPS                       0
GSW_Float                     0
GSW_MarketCap                 0
GSW_OperatingMargin           0
GSW_PE                        0
GSW_ReturnOnEq

## Convert to X and Y sequences - flat all symbols in each row

In [11]:
from numpy import array

# input and output time steps 
n_input = 16
n_out = 8

rnn_flat_x = list()
rnn_flat_y = list()

rnn_flat_test_x = list()
rnn_flat_test_y = list()

# number of features
n_features = all_x.shape[1]
#     print(n_features)

in_start = 0
eighty_num = int((len(all_x) - n_input) * .80)

# Reset y values of -999 to 0 
padding_msk = (all_y == -999)
print(len(all_y), 'y vals,', padding_msk.sum(), 'padding vals (-999)')
print('Updating -999 to 0')
all_y[padding_msk] = 0
padding_msk = (all_y == -999)
print(len(all_y), 'y vals,', padding_msk.sum(), 'padding vals (-999)')

# step over the first 80% of the history one time step at a time - training
for _ in range(eighty_num):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out

    # ensure we have enough data for this instance
    if out_end < len(all_x):
        rnn_flat_x.append(all_x[in_start:in_end])
        rnn_flat_y.append(all_y[in_end:out_end])

    # move along one time step
    in_start += 1

# step over the last 20% of the history one time step at a time - test
for _ in range(eighty_num + 1, len(all_x)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out

    # ensure we have enough data for this instance
    if out_end < len(all_x):
        rnn_flat_test_x.append(all_x[in_start:in_end])
        rnn_flat_test_y.append(all_y[in_end:out_end])

    # move along one time step
    in_start += 1
        

rnn_flat_train_x = array(rnn_flat_x)
rnn_flat_train_y = array(rnn_flat_y)

rnn_flat_test_x = array(rnn_flat_test_x)
rnn_flat_test_y = array(rnn_flat_test_y)

n_timesteps, n_features, n_outputs, out_features = rnn_flat_train_x.shape[1], \
    rnn_flat_train_x.shape[2], rnn_flat_train_y.shape[1], rnn_flat_train_y.shape[2]

# reshape output into [samples, timesteps, features]
# rnn_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], rnn_train_y.shape[1], rnn_train_y.shape[2]))

test_n_timesteps, test_n_features, test_n_outputs = rnn_flat_test_x.shape[1], \
    rnn_flat_test_x.shape[2], rnn_flat_test_y.shape[1]

# reshape output into [samples, timesteps, features]
# rnn_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], rnn_test_y.shape[1], 1))


print('-'*20)
print('Overall totals')

print(' - train timesteps', n_timesteps)
print(' - train features', n_features)
print(' - train outputs', n_outputs)

print(' - train X shape: ', rnn_flat_train_x.shape)
print(' - train y shape: ', rnn_flat_train_y.shape)

print(' - test timesteps', test_n_timesteps)
print(' - test features', test_n_features)
print(' - test outputs', test_n_outputs)

print(' - test X shape: ', rnn_flat_test_x.shape)
print(' - test y shape: ', rnn_flat_test_y.shape)

602 y vals, 14310 padding vals (-999)
Updating -999 to 0
602 y vals, 0 padding vals (-999)
--------------------
Overall totals
 - train timesteps 16
 - train features 1714
 - train outputs 8
 - train X shape:  (468, 16, 1714)
 - train y shape:  (468, 8, 113)
 - test timesteps 16
 - test features 1714
 - test outputs 8
 - test X shape:  (110, 16, 1714)
 - test y shape:  (110, 8, 113)


## Convert to X and Y sequences - separate symbol rows

In [ ]:
from numpy import array

# input and output time steps 
n_input = 16
n_out = 16

rnn_list_x = list()
rnn_list_y = list()
rnn_list_symbols = list()

rnn_list_test_x = list()
rnn_list_test_y = list()
rnn_list_test_symbols = list()


for symbol in experiment_symbols:
    # Filter to model data for this symbol and re-set the pandas indexes
    model_data = all_train_x.loc[symbol_lookup == symbol]
    
    X = list()
    y = list()
    test_X = list()
    test_y = list()
        
    # nuber of features
    n_features = model_data.shape[1]
#     print(n_features)
    
    in_start = 0
    eighty_num = int((len(model_data) - n_input) * .80)
    
    # step over the first 80% of the history one time step at a time - training
    for _ in range(eighty_num):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        
        # ensure we have enough data for this instance
        if out_end < len(model_data):
            X.append(model_data.iloc[in_start:in_end, :].values)
            y.append(model_data.iloc[in_end:out_end, n_features-1].values)
            
            rnn_list_x.append(model_data.iloc[in_start:in_end, :].values)
            rnn_list_y.append(model_data.iloc[in_end:out_end, n_features-1].values)
            rnn_list_symbols.append(symbol)
            
#             if (_ % 200 == 199):
#                 print('-'*80)
#                 print(symbol)
#                 print(model_data.iloc[in_start:in_end, :])
#                 print('-'*50)
#                 print(model_data.iloc[in_end:out_end, n_features-1])

        # move along one time step
        in_start += 1

    # step over the last 20% of the history one time step at a time - test
    for _ in range(eighty_num + 1, len(model_data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        
        # ensure we have enough data for this instance
        if out_end < len(model_data):
            test_X.append(model_data.iloc[in_start:in_end, :].values)
            test_y.append(model_data.iloc[in_end:out_end, n_features-1].values)
            
            rnn_list_test_x.append(model_data.iloc[in_start:in_end, :].values)
            rnn_list_test_y.append(model_data.iloc[in_end:out_end, n_features-1].values)
            rnn_list_test_symbols.append(symbol)
            
#             if (_ % 200 == 199):
#                 print('-'*80)
#                 print(symbol)
#                 print(model_data.iloc[in_start:in_end, :])
#                 print('-'*50)
#                 print(model_data.iloc[in_end:out_end, n_features-1])

        # move along one time step
        in_start += 1
        
    train_x = array(X)
    train_y = array(y)
    test_x = array(test_X)
    test_y = array(test_y)
    
    print('-' * 80)
    print('Symbol:', symbol)
    print(' - train sequence shape:', train_x.shape)
    print(' - test sequence shape:', test_x.shape)
    
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    
    # reshape output into [samples, timesteps, features]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    print('train n_timesteps', n_timesteps)
    print('train n_features', n_features)
    print('train n_outputs', n_outputs)
    print('-' * 5)

    test_n_timesteps, test_n_features, test_n_outputs = test_x.shape[1], test_x.shape[2], test_y.shape[1]
    
    # reshape output into [samples, timesteps, features]
    test_y = test_y.reshape((test_y.shape[0], test_y.shape[1], 1))

    print('test n_timesteps', test_n_timesteps)
    print('test n_features', test_n_features)
    print('test n_outputs', test_n_outputs)
    print('-' * 5)
    
    print('train x rnn len:', len(rnn_list_x))
    print('train y rnn len:', len(rnn_list_y))
    print('test x rnn len:', len(rnn_list_test_x))
    print('test y rnn len:', len(rnn_list_test_y))

    
#     print(train_x)
#     print(train_y)

# # Create concatenated dataframe with all data
# print('Creating concatenated dataframes')
# df_train_x = pd.concat(train_x_dfs)

rnn_train_x = array(rnn_list_x)
rnn_train_y = array(rnn_list_y)
rnn_symbols = array(rnn_list_symbols)

rnn_test_x = array(rnn_list_test_x)
rnn_test_y = array(rnn_list_test_y)
rnn_test_symbols = array(rnn_list_test_symbols)


n_timesteps, n_features, n_outputs = rnn_train_x.shape[1], rnn_train_x.shape[2], rnn_train_y.shape[1]

# reshape output into [samples, timesteps, features]
rnn_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], rnn_train_y.shape[1], 1))

test_n_timesteps, test_n_features, test_n_outputs = rnn_test_x.shape[1], rnn_test_x.shape[2], rnn_test_y.shape[1]

# reshape output into [samples, timesteps, features]
rnn_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], rnn_test_y.shape[1], 1))


print('-'*20)
print('Overall totals')

print(' - train timesteps', n_timesteps)
print(' - train features', n_features)
print(' - train outputs', n_outputs)

print(' - train X shape: ', rnn_train_x.shape)
print(' - train y shape: ', rnn_train_y.shape)
print(' - train symbols lookup shape: ', rnn_symbols.shape)

print(' - test timesteps', test_n_timesteps)
print(' - test features', test_n_features)
print(' - test outputs', test_n_outputs)

print(' - test X shape: ', rnn_test_x.shape)
print(' - test y shape: ', rnn_test_y.shape)
print(' - test symbols lookup shape: ', rnn_test_symbols.shape)

In [ ]:
rnn_train_x.shape
# print(df_train_x.drop(['future_eight_week_return'], axis=1, errors='ignore').columns.values)

In [ ]:
rnn_train_y.shape

In [ ]:
rnn_train_y.reshape(len(rnn_train_y),4)

In [ ]:
print(rnn_train_x[0, :, 247])
print(rnn_train_x[1, :, 247])
print(rnn_train_x[2, :, 247])
print(rnn_train_x[3, :, 247])

In [ ]:
print(rnn_train_y[0])
print(rnn_train_y[1])
print(rnn_train_y[2])
print(rnn_train_y[3])

In [ ]:
print(np.unique(rnn_symbols))
msk = rnn_symbols=='ALT'
print(msk)

In [ ]:
def plot_prediction(x, y_true, y_pred):
    """Plots the predictions.
    
    Arguments
    ---------
    x: Input sequence of shape (input_sequence_length,
        dimension_of_signal)
    y_true: True output sequence of shape (input_sequence_length,
        dimension_of_signal)
    y_pred: Predicted output sequence (input_sequence_length,
        dimension_of_signal)
    """

    plt.figure(figsize=(12, 3))

    output_dim = x.shape[-1]
    for j in range(output_dim):
        past = x[:, j] 
        true = y_true[:, j]
        pred = y_pred[:, j]

        label1 = "Seen (past) values" if j==0 else "_nolegend_"
        label2 = "True future values" if j==0 else "_nolegend_"
        label3 = "Predictions" if j==0 else "_nolegend_"

        plt.plot(range(len(past)), past, "o--b",
                 label=label1)
        plt.plot(range(len(past),
                 len(true)+len(past)), true, "x--b", label=label2)
        plt.plot(range(len(past), len(pred)+len(past)), pred, "o--y",
                 label=label3)
    plt.legend(loc='best')
    plt.title("Predictions v.s. true values")
    plt.show()

In [ ]:
## Reference data
from scipy import stats

print('Min:', np.min(rnn_train_y))
print('Max:', np.max(rnn_train_y))
print('Median:', np.median(rnn_train_y))
print('Modes:', stats.mode(rnn_train_y))
print('Averages:', np.mean(rnn_train_y))
print('25th percentiles:', np.percentile(rnn_train_y, 25))
print('80th percentiles:', np.percentile(rnn_train_y, 80))
print('90th percentile:', np.percentile(rnn_train_y, 90))


for symbol in np.unique(rnn_symbols):
    print('-'*80)
    print(symbol)
    print('-'*80)
    msk = (rnn_symbols=='ALT')
    print('Min:', np.min(rnn_train_y[msk]))
    print('Max:', np.max(rnn_train_y[msk]))
    print('Median:', np.median(rnn_train_y[msk]))
    print('Modes:', stats.mode(rnn_train_y[msk]))
    print('Averages:', np.mean(rnn_train_y[msk]))
    print('25th percentiles:', np.percentile(rnn_train_y[msk], 25))
    print('80th percentiles:', np.percentile(rnn_train_y[msk], 80))
    print('90th percentile:', np.percentile(rnn_train_y[msk], 90))

## Model evaluator

In [12]:
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score, median_absolute_error
from stats_operations import *


def eval_results(actual_y, predicted_y):
    num_vals = len(actual_y)
    # Work through each val in y
    mae = mean_absolute_error(actual_y, predicted_y)
    mape = safe_mape(actual_y, predicted_y)
    cmape = clipped_mape(actual_y, predicted_y)
    smape = symmetrical_mape(actual_y, predicted_y)
    ae_p25 = abs_err_p(actual_y, predicted_y, 25)
    ae_p50 = abs_err_p(actual_y, predicted_y, 50)
    ae_p75 = abs_err_p(actual_y, predicted_y, 75)
    ae_p90 = abs_err_p(actual_y, predicted_y, 90)
    ape_p75 = safe_mape_p(actual_y, predicted_y, 75)
    ape_p90 = safe_mape_p(actual_y, predicted_y, 90)    
    rsquared = r2_score(actual_y, predicted_y)
    explain_variance = explained_variance_score(actual_y, predicted_y)

    return mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, explain_variance, \
            ape_p75, ape_p90, num_vals

def perform_eval(name, actuals, predictions):
    eval_dict = {
        'name': [name],
        'num_vals': [len(actuals)]
    }
    
    maes = []
    mapes = []
    cmapes = []
    smapes = []
    ae_p25s = []
    ae_p50s = []
    ae_p75s = []
    ae_p90s = []
    rsquareds = []
    explain_variances = []

    # execute eval per column
    for _ in range(actuals.shape[1]):
        col_actual_y = actuals[:, _]
        col_predict_y = predictions[:, _]
       
        mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, \
        explain_variance, num_vals = eval_results(col_actual_y, col_predict_y)

        mean_actual_val = np.mean(col_actual_y)
        median_actual_val = np.median(col_actual_y)

        mean_predicted_val = np.mean(col_predict_y)
        median_predicted_val = np.median(col_predict_y)

        eval_dict['y_' + str(_) + '_mean_actual_val'] = [mean_actual_val]
        eval_dict['y_' + str(_) + '_median_actual_val'] = [median_actual_val]
        eval_dict['y_' + str(_) + '_mean_predicted_val'] = [mean_predicted_val]
        eval_dict['y_' + str(_) + '_median_predicted_val'] = [median_predicted_val]
        eval_dict['y_' + str(_) + '_mae'] = [mae] 
        eval_dict['y_' + str(_) + '_mape'] = [mape]
        eval_dict['y_' + str(_) + '_cmape'] = [cmape]
        eval_dict['y_' + str(_) + '_smape'] = [smape]
        eval_dict['y_' + str(_) + '_abs_err_p25'] = [ae_p25]
        eval_dict['y_' + str(_) + '_abs_err_p50'] = [ae_p50]
        eval_dict['y_' + str(_) + '_abs_err_p75'] = [ae_p75]
        eval_dict['y_' + str(_) + '_abs_err_p90'] = [ae_p90]
        eval_dict['y_' + str(_) + '_rsquared'] = [rsquared]
        eval_dict['y_' + str(_) + '_explain_variance'] = [explain_variance]
        
        maes.append(mae)
        mapes.append(mape)
        cmapes.append(cmape)
        smapes.append(smape)
        ae_p25s.append(ae_p25)
        ae_p50s.append(ae_p50)
        ae_p75s.append(ae_p75)
        ae_p90s.append(ae_p90)
        rsquareds.append(rsquared)
        explain_variances.append(explain_variance)
        
    eval_dict['all_mae'] = np.mean(maes)
    eval_dict['all_mape'] = np.mean(mapes)
    eval_dict['all_cmape'] = np.mean(cmapes)
    eval_dict['all_smape'] = np.mean(smapes)
    eval_dict['all_abs_err_p25'] = np.mean(ae_p25s)
    eval_dict['all_abs_err_p50'] = np.mean(ae_p50s)
    eval_dict['all_abs_err_p75'] = np.mean(ae_p75s)
    eval_dict['all_abs_err_p90'] = np.mean(ae_p90s)
    eval_dict['all_rsquared'] = np.mean(rsquareds)
    eval_dict['all_explain_variance'] = np.mean(explain_variances)
        
#     print(eval_dict)
    eval_df = pd.DataFrame.from_dict(eval_dict)
    
    return eval_df

def evaluate_results(symbols_x, predictions, actuals):
    # Determine unique list of symbols
    symbols = np.unique(symbols_x)

    print('Overall results, number of items in prediction data:', len(predictions))
    df_results = perform_eval('overall', actuals, predictions)
    
    print('Executing symbol results, number of symbols in prediction data:', len(symbols))

    for symbol in symbols:
        # Retrieve data indices which match symbols
        pred_index = np.where(symbols_x == symbol)[0]

        # Retrieve data which matches symbol
        symbol_predictions = predictions[pred_index]
        symbol_actuals = actuals[pred_index]
        
        print(symbol, 'results, number of items in prediction data:', len(symbol_predictions))

        df_symbol_results = perform_eval(symbol, symbol_actuals, symbol_predictions)

        # Add data frame into all results
        df_results = pd.concat([df_results, df_symbol_results])

    # output overall summary
    means = ['all_mae', 'all_mape', 'all_cmape', 'all_smape', 'all_abs_err_p25', 
             'all_abs_err_p50', 'all_abs_err_p75', 'all_abs_err_p90', 
             'all_rsquared', 'all_explain_variance']
    print(df_results[df_results['name']=='overall'][means])
        
    return df_results

In [13]:
def eval_flat_predictions(y_true, y_pred,target_col_list):
    result = {}
    # Work through overall 
    y_true_overall = y_true.flatten()
    y_pred_overall = y_pred.flatten()    
    mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, \
            explain_variance, ape_p75, ape_p90, num_vals = eval_results(y_true_overall, y_pred_overall)
    result = {
        "name": ["Overall"],
        'mean_y': [np.mean(y_true_overall)],
        'mae': [mae],
        'mape': [mape],
        'cmape': [cmape],
        'ae_p75': [ae_p75],
        'ae_p90': [ae_p90],
        'ape_p75': [ape_p75],
        'ape_p90': [ape_p90]
    }

        
    # Overall per output step
    print('Overall results per week')
    print('-' * 40)
    for step in range(y_true.shape[1]):
        print('Week', step+1)

        y_true_week = y_true[:, step].flatten()
        y_pred_week = y_pred[:, step].flatten()
        mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, \
            explain_variance, ape_p75, ape_p90, num_vals = eval_results(y_true_week, y_pred_week)

        result[str(step+1) + '_mean_y'] = [np.mean(y_true_week)]
        result[str(step+1) + '_mae'] = [mae]
        result[str(step+1) + '_mape'] = [mape]
        result[str(step+1) + '_cmape'] = [cmape]
        result[str(step+1) + '_ae_p75'] = [ae_p75]
        result[str(step+1) + '_ae_p90'] = [ae_p90]
        result[str(step+1) + '_ape_p75'] = [ape_p75]
        result[str(step+1) + '_ape_p90'] = [ape_p90]
    
    # create data frame from results
    df_results = pd.DataFrame.from_dict(result)

            
    
    # Each symbol overall
    print('-' * 40)
    print('Results per target')
    for col_num in range(len(target_col_list)):
        target_name = target_col_list[col_num]
        print(" ", target_name)
        y_true_week = y_true[:, :, col_num].flatten()
        y_pred_week = y_pred[:, :, col_num].flatten()
        mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, \
            explain_variance, ape_p75, ape_p90, num_vals = eval_results(y_true_week, y_pred_week)

        result = {
            "name": [target_name],
            'mean_y': [np.mean(y_true_week)],
            'mae': [mae],
            'mape': [mape],
            'cmape': [cmape],
            'ae_p75': [ae_p75],
            'ae_p90': [ae_p90],
            'ape_p75': [ape_p75],
            'ape_p90': [ape_p90]
        }


        for step in range(y_true.shape[1]):
            print('    Week', step+1)
            y_true_week = y_true[:, step, col_num].flatten()
            y_pred_week = y_pred[:, step, col_num].flatten()
            mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, \
                explain_variance, ape_p75, ape_p90, num_vals = eval_results(y_true_week, y_pred_week)

            result[str(step+1) + '_mean_y'] = [np.mean(y_true_week)]
            result[str(step+1) + '_mae'] = [mae]
            result[str(step+1) + '_mape'] = [mape]
            result[str(step+1) + '_cmape'] = [cmape]
            result[str(step+1) + '_ae_p75'] = [ae_p75]
            result[str(step+1) + '_ae_p90'] = [ae_p90]
            result[str(step+1) + '_ape_p75'] = [ape_p75]
            result[str(step+1) + '_ape_p90'] = [ape_p90]

        # create data frame from results
        df_current_result = pd.DataFrame.from_dict(result)

        # Add data frame into all results
        df_results = pd.concat([df_results, df_current_result])
        

    
    return df_results

## Time Distributed LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM, GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.regularizers import L1L2
from keras import optimizers
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from AdamW import AdamW
from stats_operations import *
from chrono_lstm import ChronoLSTM

# rnn_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], n_outputs, 1))
# rnn_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], n_outputs, 1))

layers = [2, 2, 2, 2, 2, 2]
batch_size = 128
num_epochs = 200
num_samples = rnn_flat_train_x.shape[0]

scaled_layers = []
for layer in layers:
    scaled_layers.append(max(int(n_features * layer), 1))

early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# csv_logger = CSVLogger('./logs/nn-training-log.log')
checkpointer = ModelCheckpoint(
filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5, verbose=1)
weight_decay = 0.005 * batch_size / num_samples / num_epochs **0.5

print(scaled_layers)
print(batch_size)
print(num_epochs)
print(num_samples)
print(weight_decay)
print('input steps', n_timesteps)
print('output steps', n_outputs)

# define model
model = Sequential()
# model.add(ChronoLSTM(scaled_layers[0], 
#                     (num_epochs*n_timesteps),
#                    activation='relu', 
#                    input_shape=(n_timesteps, n_features)
#                     ))
model.add(GRU(scaled_layers[0], 
                   activation='relu', 
                   input_shape=(n_timesteps, n_features)
#                    dropout=0.2
#                    kernel_regularizer=L1L2(0.0001, 0.0001)
          ))
model.add(RepeatVector(n_outputs))

# for layer_size in scaled_layers[1:-1]:
# model.add(ChronoLSTM(scaled_layers[0], 
#                      num_epochs*n_timesteps, 
#                      activation='relu', 
#                      return_sequences=True))

model.add(GRU(scaled_layers[1], 
               activation='relu', 
#                dropout=0.2,
               return_sequences=True))
model.add(TimeDistributed(Dense(scaled_layers[1], activation='relu')))

model.add(GRU(scaled_layers[2], 
               activation='relu', 
#                dropout=0.2,
               return_sequences=True))
model.add(TimeDistributed(Dense(scaled_layers[1], activation='relu')))

model.add(GRU(scaled_layers[3], 
               activation='relu', 
#                dropout=0.2,
               return_sequences=True))

model.add(GRU(scaled_layers[3], 
               activation='relu', 
#                dropout=0.2,
               return_sequences=True))

model.add(GRU(scaled_layers[5], 
               activation='relu', 
#                dropout=0.2,
               return_sequences=True))
model.add(TimeDistributed(Dense(scaled_layers[2], activation='relu')))
model.add(TimeDistributed(Dense(rnn_flat_train_y.shape[2])))

# AdamW(weight_decay=weight_decay)
model.compile(loss='mae', optimizer=optimizers.RMSprop(lr=0.002), metrics=[k_clipped_mape])
print(model.summary())

# fit network
model.fit(rnn_flat_train_x, 
          rnn_flat_train_y,
          validation_data = (rnn_flat_test_x, rnn_flat_test_y),
          epochs=num_epochs, 
          batch_size=batch_size, 
          callbacks=[early_stopping, checkpointer, reduce_lr],
          verbose=1)

model.load_weights('./weights/nn-weights.hdf5')
y_flat_test_predictions = model.predict(rnn_flat_test_x)

# model_eval = evaluate_results(rnn_test_symbols, y_test_predictions, rnn_test_y)
# model_eval.to_csv('./results/janet-time-distributed-%d-%d-%d-epochs.csv' %(n_input, n_out, num_epochs))

y_true = rnn_flat_test_y.flatten()
y_pred = y_flat_test_predictions.flatten()    
mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, \
        explain_variance, ape_p75, ape_p90, num_vals = eval_results(y_true, y_pred)
print('num vals:', num_vals)
print('mae:', mae)
print('mape:', mape)
print('cmape:', cmape)
print('ae_p75:', ae_p75)
print('ae_p90:', ae_p90)
print('ape_p75:', ape_p75)
print('ape_p90:', ape_p90)

df_results = eval_flat_predictions(rnn_flat_test_y, y_flat_test_predictions, target_col_names)
df_results.to_csv('./results/time-distributed.csv')

print('Done')
# model_eval

# 16-4, 100 epochs: loss: 8.0383 - val_loss: 5.6601
# 16-4, 200 epochs: loss: 7.9807 - val_loss: 5.6143
# 12-4, 100 epochs: loss: 7.9533 - val_loss: 5.8089
# 20-4, 200 epochs: loss: 8.2571 - val_loss: 6.0177
# 8-4, 200 epochs: loss: 7.9358 - val_loss: 5.7033
# 16-4, 200 epochs, L1L2 0.0001:
# 16-4, 200 epochs, GRU, L1L2 0.001, mae: 5.704350709915161, mape: 72.01476097106934, RMSprop: 2.718229293823242

## 57 symbols in 1 row
# 16-8, 200 epochs LSTM mape adam,
#                         mae: 3.0469394
#                         mape: 62.94738054275513
#                         cmape: 67.20255613327026
#                         smape: 198.20615296059964
#                         ae_p25: 0.03171489853411913
#                         ae_p50: 1.1797866225242615
#                         ae_p75: 3.9442883133888245
#                         ae_p90: 7.953948116302489
# 16-8, 200 epochs LSTM mae adam,
#                         mae: 3.037885
#                         mape: 62.2211754322052
#                         cmape: 65.83806872367859
#                         smape: 198.04303329443397
#                         ae_p25: 0.006103984662331641
#                         ae_p50: 1.176807165145874
#                         ae_p75: 3.981552243232727
#                         ae_p90: 7.9905468940734865
# 16-8, 200 epochs LSTM mae AdamW,
#                         mae: 3.0392604
#                         mape: 62.26362586021423
#                         cmape: 65.8830463886261
#                         smape: 198.74643603440208
#                         ae_p25: 0.00881427968852222
#                         ae_p50: 1.1775119304656982
#                         ae_p75: 3.969123423099518
#                         ae_p90: 7.973479557037353
# 16-8, 200 epochs LSTM mae RMSprop,
#                         mae: 3.0371118
#                         mape: 62.067198753356934
#                         cmape: 65.49670696258545
#                         smape: 198.74755064508935
#                         ae_p25: 0.0002485034929122776
#                         ae_p50: 1.176451325416565
#                         ae_p75: 3.9679842591285706
#                         ae_p90: 7.998835563659668
# 16-8, 200 epochs LSTM mae RMSprop (4, 4 layers),
#                         mae: 3.0378606
#                         mape: 62.15248107910156
#                         cmape: 65.67736864089966
#                         smape: 198.5776040141972
#                         ae_p25: 0.00444786436855793
#                         ae_p50: 1.1738138794898987
#                         ae_p75: 3.979398727416992
#                         ae_p90: 7.9832131385803216
# 16-8, 200 epochs LSTM mae RMSprop (2, 2, 2 layers),
#                         mae: 3.037165
#                         mape: 62.07066774368286
#                         cmape: 65.5011773109436
#                         smape: 198.75001840814292
#                         ae_p25: 3.774190554395318e-05
#                         ae_p50: 1.1764426827430725
#                         ae_p75: 3.9704267382621765
#                         ae_p90: 7.99989161491394
# 16-8, 200 epochs GRU mae RMSprop (2, 2, 2 layers),
#                         mae: 3.036734
#                         mape: 62.17632293701172
#                         cmape: 65.78048467636108
#                         smape: 197.84315421789358
#                         ae_p25: 0.001321911724517122
#                         ae_p50: 1.1769254803657532
#                         ae_p75: 3.9657304286956787
#                         ae_p90: 7.975777339935299
# 16-8, 200 epochs GRU mae RMSprop (2, 2, 2, 2 layers),
#                         mae: 3.0370674
#                         mape: 62.087345123291016
#                         cmape: 65.54595828056335
#                         smape: 198.62712994060067
#                         ae_p25: 0.0003695989726111293
#                         ae_p50: 1.1766981482505798
#                         ae_p75: 3.9665741324424744
#                         ae_p90: 7.995802545547484
# 16-8, 200 epochs GRU mae RMSprop (2, 2, 2, 2, 2 layers),
#                         mae: 3.0367754
#                         mape: 62.0708703994751
#                         cmape: 65.51564931869507
#                         smape: 198.7152693335687
#                         ae_p25: 0.00028848367219325155
#                         ae_p50: 1.1766218543052673
#                         ae_p75: 3.9676144123077393
#                         ae_p90: 7.9992693901062015
# 16-8, 200 epochs GRU mae RMSprop(lr=0.005) (2, 2, 2, 2, 2, 2 layers),
#                         mae: 3.0516613
#                         mape: 63.53962421417236
#                         cmape: 67.20293760299683
#                         smape: 196.93598114939047
#                         ae_p25: 0.015180134447291493
#                         ae_p50: 1.2153788208961487
#                         ae_p75: 3.972410023212433
#                         ae_p90: 7.973083543777466
# 16-8, 200 epochs GRU mae RMSprop(lr=0.002) (2, 2, 2, 2, 2, 2 layers),


## Seq2Seq Encoder - Decoder

In [21]:
from keras.layers.core import *
from keras.layers.recurrent import LSTM, GRU, SimpleRNN, RNN
from keras.layers import Bidirectional, Flatten, RepeatVector, TimeDistributed, Concatenate
from keras.regularizers import L1L2
from keras.models import *
from keras import optimizers
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from AdamW import AdamW
from stats_operations import *
from attention import AttentionLayer

# rnn_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], n_outputs))
# rnn_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], n_outputs))

encoder_layers = [2, 2, 1, 1]
decoder_layers = [1, 1, 2, 2]
batch_size = 128
num_epochs = 200
num_samples = rnn_flat_train_x.shape[0]
n_timesteps = rnn_flat_train_x.shape[1]
n_features = rnn_flat_train_x.shape[2]
n_output_timesteps = rnn_flat_train_y.shape[1]
n_output_features = rnn_flat_train_y.shape[2]
steps_per_epoch = int(num_samples / batch_size)

scaled_encoder_layers = []
for layer in encoder_layers:
    scaled_encoder_layers.append(max(int(n_features * layer), 1))

scaled_decoder_layers = []
for layer in decoder_layers:
    scaled_decoder_layers.append(max(int(n_features * layer), 1))

print('encoder layers', scaled_encoder_layers)
print('decoder layers', scaled_decoder_layers)
print('batch_size', batch_size)
print('epochs', num_epochs)
print('Train shape')
print(num_samples, ',', n_timesteps, ',', n_features)
print(rnn_flat_train_x.shape)
print('Test shape')
print(num_samples, ',', n_output_timesteps, ', ', n_output_features)
print(rnn_flat_train_y.shape)
print('input steps', n_timesteps)
print('input features', n_features)
print('output steps', n_output_timesteps)
print('output features', n_output_features)

early_stopping = EarlyStopping(monitor='val_loss', patience=30)
checkpointer = ModelCheckpoint(
filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5, verbose=1)
weight_decay = 0.005 * batch_size / num_samples / num_epochs **0.5

encoder_inputs = Input(shape=(n_timesteps, n_features))
input_layer = GRU(scaled_encoder_layers[0], activation='relu')(encoder_inputs)
repeated_input = RepeatVector(n_outputs)(input_layer)
layer_2 = GRU(scaled_encoder_layers[1], activation='relu', return_sequences=True)(repeated_input)
layer_2_time_distributed = TimeDistributed(Dense(scaled_encoder_layers[1], activation='relu'))(layer_2)
# layer_3 = GRU(scaled_encoder_layers[2], activation='relu', return_sequences=True)(layer_2_time_distributed)
# layer_3_time_distributed = TimeDistributed(Dense(scaled_encoder_layers[2], activation='relu'))(layer_3)
final_encoder_layer, encoder_state = GRU(scaled_encoder_layers[3], activation='relu', return_sequences=True, return_state=True)(layer_2_time_distributed)

decoder_inputs = Input(shape=(n_output_timesteps, n_output_features))
decoder_layer_1, decoder_state = GRU(scaled_decoder_layers[0], 
                                     activation='relu', 
                                     return_sequences=True, 
                                     return_state=True)(decoder_inputs, initial_state=encoder_state)
decoder_layer_1_time_distributed = TimeDistributed(Dense(scaled_decoder_layers[0], activation='relu'))(decoder_layer_1)
decoder_layer_2 = GRU(scaled_decoder_layers[1], activation='relu', return_sequences=True)(decoder_layer_1_time_distributed)
decoder_layer_2_time_distributed = TimeDistributed(Dense(scaled_decoder_layers[1], activation='relu'))(decoder_layer_2)
decoder_layer_3 = GRU(scaled_decoder_layers[2], activation='relu', return_sequences=True)(decoder_layer_2_time_distributed)
decoder_layer_3_time_distributed = TimeDistributed(Dense(scaled_decoder_layers[2], activation='relu'))(decoder_layer_3)
# decoder_layer_4 = GRU(scaled_decoder_layers[3], activation='relu', return_sequences=True)(decoder_layer_3_time_distributed)
# decoder_layer_4_time_distributed = TimeDistributed(Dense(scaled_decoder_layers[3], activation='relu'))(decoder_layer_4)
decoder_output_layer = TimeDistributed(Dense(n_output_features))(decoder_layer_3_time_distributed)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_output_layer)
model.compile(loss='mae', optimizer=optimizers.RMSprop(lr=0.002), metrics=[k_mean_absolute_percentage_error, k_mae_mape])
print(model.summary())
                            
decoder_train_input = K.zeros(rnn_flat_train_y.shape)
decoder_test_input = K.zeros(rnn_flat_test_y.shape)
    
# fit network
model.fit([rnn_flat_train_x, decoder_train_input], rnn_flat_train_y,
           validation_data = ([rnn_flat_test_x, decoder_test_input], rnn_flat_test_y),
           epochs=num_epochs, 
           steps_per_epoch=1, 
           validation_steps=1,
           callbacks=[early_stopping, checkpointer, reduce_lr],
           verbose=1)

# print(encoder_state.shape)
# print(encoder_state)

model.load_weights('./weights/nn-weights.hdf5')
y_flat_test_predictions = model.predict([rnn_flat_test_x, decoder_test_input], steps=1)

y_true = rnn_flat_test_y.flatten()
y_pred = y_flat_test_predictions.flatten()    
mae, mape, cmape, smape, ae_p25, ae_p50, ae_p75, ae_p90, rsquared, \
        explain_variance, ape_p75, ape_p90, num_vals = eval_results(y_true, y_pred)
print('num vals:', num_vals)
print('mae:', mae)
print('mape:', mape)
print('cmape:', cmape)
print('ae_p75:', ae_p75)
print('ae_p90:', ae_p90)
print('ape_p75:', ae_p75)
print('ape_p90:', ae_p90)

df_results = eval_flat_predictions(rnn_flat_test_y, y_flat_test_predictions, target_col_names)
df_results.to_csv('./results/seq2seq.csv')

print('Done')

#2.5288 3 layer
#2.5293 2 layer
#2.5288 4 layer

encoder layers [3428, 3428, 1714, 1714]
decoder layers [1714, 1714, 3428, 3428]
batch_size 128
epochs 200
Train shape
468 , 16 , 1714
(468, 16, 1714)
Test shape
468 , 8 ,  113
(468, 8, 113)
input steps 16
input features 1714
output steps 8
output features 113
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 16, 1714)     0                                            
__________________________________________________________________________________________________
gru_36 (GRU)                    (None, 3428)         52890612    input_13[0][0]                   
__________________________________________________________________________________________________
repeat_vector_7 (RepeatVector)  (None, 8, 3428)      0           gru_36[0][0]                     
_______________________________________________

/Users/neilkloot/anaconda/lib/python3.6/site-packages/keras/engine/network.py:888: UserWarning: Layer gru_40 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'gru_39/while/Exit_3:0' shape=(?, 1714) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/200
1/1 [==============================] - 60s 60s/step - loss: 6.1085 - k_mean_absolute_percentage_error: 249.1408 - k_mae_mape: 49.5902 - val_loss: 1440459.6250 - val_k_mean_absolute_percentage_error: 100956576.0000 - val_k_mae_mape: 6025924050944.0000
Epoch 3/200
1/1 [==============================] - 63s 63s/step - loss: 1687020.2500 - k_mean_absolute_percentage_error: 114424632.0000 - k_mae_mape: 8892883927040.0000 - val_loss: 251131488.0000 - val_k_mean_absolute_percentage_error: 17509703680.0000 - val_k_mae_mape: 209402934402744320.0000
Epoch 4/200
1/1 [==============================] - 77s 77s/step - loss: 344384480.0000 - k_mean_absolute_percentage_error: 22665371648.0000 - k_mae_mape: 389567037124378624.0000 - val_loss: 4.7167 - val_k_mean_absolute_percentage_error: 254.2543 - val_k_mae_mape: 34.7143
Epoch 5/200
1/1 [==============================] - 70s 70s/step - loss: 6.9156 - k_mean_absolute_percentage_error: 300.9145 - k_mae_mape: 63.3765 - val_loss: 315.3219 - v

Epoch 36/200
1/1 [==============================] - 73s 73s/step - loss: 4.1062 - k_mean_absolute_percentage_error: 52.7171 - k_mae_mape: 2.2815 - val_loss: 2.5293 - val_k_mean_absolute_percentage_error: 57.2612 - val_k_mae_mape: 1.5969
Epoch 37/200
1/1 [==============================] - 74s 74s/step - loss: 4.1062 - k_mean_absolute_percentage_error: 52.7202 - k_mae_mape: 2.2823 - val_loss: 2.5293 - val_k_mean_absolute_percentage_error: 57.2588 - val_k_mae_mape: 1.5967
Epoch 38/200
1/1 [==============================] - 74s 74s/step - loss: 4.1062 - k_mean_absolute_percentage_error: 52.7174 - k_mae_mape: 2.2815 - val_loss: 2.5293 - val_k_mean_absolute_percentage_error: 57.2575 - val_k_mae_mape: 1.5965
Epoch 39/200
1/1 [==============================] - 74s 74s/step - loss: 4.1062 - k_mean_absolute_percentage_error: 52.7171 - k_mae_mape: 2.2817 - val_loss: 2.5294 - val_k_mean_absolute_percentage_error: 57.2639 - val_k_mae_mape: 1.5973

Epoch 00039: ReduceLROnPlateau reducing learning ra

Epoch 70/200
1/1 [==============================] - 73s 73s/step - loss: 4.1057 - k_mean_absolute_percentage_error: 52.6717 - k_mae_mape: 2.2741 - val_loss: 2.5290 - val_k_mean_absolute_percentage_error: 57.2261 - val_k_mae_mape: 1.5932
Epoch 71/200
1/1 [==============================] - 71s 71s/step - loss: 4.1058 - k_mean_absolute_percentage_error: 52.6740 - k_mae_mape: 2.2745 - val_loss: 2.5290 - val_k_mean_absolute_percentage_error: 57.2255 - val_k_mae_mape: 1.5932

Epoch 00071: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 72/200
1/1 [==============================] - 73s 73s/step - loss: 4.1058 - k_mean_absolute_percentage_error: 52.6731 - k_mae_mape: 2.2744 - val_loss: 2.5289 - val_k_mean_absolute_percentage_error: 57.2193 - val_k_mae_mape: 1.5926
Epoch 73/200
1/1 [==============================] - 71s 71s/step - loss: 4.1057 - k_mean_absolute_percentage_error: 52.6645 - k_mae_mape: 2.2729 - val_loss: 2.5289 - val_k_mean_absolute_percentage_error: 57.2

Epoch 104/200
1/1 [==============================] - 72s 72s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6579 - k_mae_mape: 2.2718 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2143 - val_k_mae_mape: 1.5920
Epoch 105/200
1/1 [==============================] - 71s 71s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6580 - k_mae_mape: 2.2719 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2143 - val_k_mae_mape: 1.5920
Epoch 106/200
1/1 [==============================] - 71s 71s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6582 - k_mae_mape: 2.2719 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2145 - val_k_mae_mape: 1.5920
Epoch 107/200
1/1 [==============================] - 71s 71s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6582 - k_mae_mape: 2.2719 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2141 - val_k_mae_mape: 1.5920
Epoch 108/200
1/1 [=============================

Epoch 138/200
1/1 [==============================] - 69s 69s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6568 - k_mae_mape: 2.2717 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2135 - val_k_mae_mape: 1.5920
Epoch 139/200
1/1 [==============================] - 69s 69s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6569 - k_mae_mape: 2.2717 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2136 - val_k_mae_mape: 1.5920
Epoch 140/200
1/1 [==============================] - 70s 70s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6569 - k_mae_mape: 2.2717 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2135 - val_k_mae_mape: 1.5919
Epoch 141/200
1/1 [==============================] - 70s 70s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6569 - k_mae_mape: 2.2717 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2136 - val_k_mae_mape: 1.5920
Epoch 142/200
1/1 [=============================

Epoch 173/200
1/1 [==============================] - 69s 69s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6565 - k_mae_mape: 2.2716 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2134 - val_k_mae_mape: 1.5919
Epoch 174/200
1/1 [==============================] - 69s 69s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6565 - k_mae_mape: 2.2716 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2134 - val_k_mae_mape: 1.5919
Epoch 175/200
1/1 [==============================] - 69s 69s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6565 - k_mae_mape: 2.2716 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2134 - val_k_mae_mape: 1.5919
Epoch 176/200
1/1 [==============================] - 69s 69s/step - loss: 4.1056 - k_mean_absolute_percentage_error: 52.6566 - k_mae_mape: 2.2716 - val_loss: 2.5288 - val_k_mean_absolute_percentage_error: 57.2134 - val_k_mae_mape: 1.5919
Epoch 177/200
1/1 [=============================

Week 4
Week 5
Week 6
Week 7
Week 8
DOW
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
NVL
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
AOG
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
BGL
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
HSN
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
CSS
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
SWM
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
RIC
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
CMI
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
MKE
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
BCI
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
IBG
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
SWJ
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
IXP
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
MYX
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
RNY
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
CZA
W

In [ ]:
df_results = eval_flat_predictions(rnn_flat_test_y, y_flat_test_predictions, target_col_names)
df_results.to_csv('./results/seq2seq.csv')


## Basic LSTM 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM, GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from AdamW import AdamW

rnn_train_y = rnn_train_y.reshape(len(rnn_train_y), 4)
rnn_test_y = rnn_test_y.reshape(len(rnn_test_y), 4)

layers = [2, 2, 1]
batch_size = 128
num_epochs = 50
num_samples = rnn_train_x.shape[0]
n_timesteps =  rnn_train_x.shape[1]
n_features = rnn_train_x.shape[2]
n_outputs = rnn_train_y.shape[1]

scaled_layers = []
for layer in layers:
    scaled_layers.append(max(int(n_features * layer), 1))

    
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# csv_logger = CSVLogger('./logs/nn-training-log.log')
checkpointer = ModelCheckpoint(
filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5, verbose=1)
weight_decay = 0.005 * batch_size / num_samples / num_epochs **0.5

print(scaled_layers)
print(batch_size)
print(num_epochs)
print(num_samples)
print(n_timesteps)
print(n_features)
print(n_outputs)
print(weight_decay)


# define model
model2 = Sequential()
model2.add(LSTM(scaled_layers[0], activation='relu', 
                input_shape=(n_timesteps, n_features), 
                kernel_regularizer=L1L2(0.001, 0.001),
                return_sequences=True))
# model2.add(LSTM(scaled_layers[1], activation='relu', return_sequences=True))
model2.add(Dense(scaled_layers[2], activation='relu'))
model2.add(Dense(n_outputs))
# model2.compile(loss='mae', optimizer='adam')
model2.compile(loss='mean_squared_logarithmic_error', optimizer=AdamW(weight_decay=weight_decay))

# fit network
model2.fit(rnn_train_x, 
          rnn_train_y,
#           validation_split=0.2,
          validation_data = (rnn_test_x, rnn_test_y),
          epochs=num_epochs, 
          batch_size=batch_size, 
          callbacks=[early_stopping, checkpointer, reduce_lr],
          verbose=1)

model2.load_weights('./weights/nn-weights.hdf5')
y_test_predictions = model2.predict(rnn_test_x)

model_eval = evaluate_results(rnn_test_symbols, y_test_predictions, rnn_test_y)
model_eval.to_csv('./results/lstm-%d-%d-%d-epochs-adamw-l1l2-001.csv'%(n_input, n_out, num_epochs))
print('Done')
model_eval

# 12-4, 100 epochs, L1L2 0.01: loss: 7.9516 - val_loss: 5.8052
# 12-4, 200 epochs, L1L2 0.01: loss: 7.8844 - val_loss: 5.7364
# 16-4, 200 epochs, L1L2 0.01: loss: 7.9167 - val_loss: 5.5488
# 20-4, 200 epochs, L1L2 0.01: loss: 8.3992 - val_loss: 6.1353
# 8-4, 200 epochs, L1L2 0.01: loss: 7.9486 - val_loss: 5.7188
# 8-4, 200 epochs, L1L2 0.008: loss: 7.9347 - val_loss: 5.7033
# 8-4, 200 epochs, L1L2 0.005: loss: 7.9348 - val_loss: 5.7030
# 16-4, 200 epochs, L1L2 0.001, AdamW: loss: 7.9842 - val_loss: 5.7384

In [ ]:
print('Executing keras predictions...')
print('Data: rnn_test_x', rnn_test_x.shape)
print('Data: rnn_test_y', rnn_test_y.shape)
print('Data: rnn_test_symbols', rnn_test_symbols.shape)

y_test_predictions = model2.predict(rnn_test_x)
print('Predictions: ', y_test_predictions.shape)


model_eval = evaluate_results(rnn_test_symbols, y_test_predictions, rnn_test_y)
model_eval.to_csv('./results/lstm-16-4-200-epochs.csv')
print('Done')
model_eval

## CNN LSTM

In [ ]:
## Time Distributed LSTM

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.regularizers import L1L2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from AdamW import AdamW

rnn_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], 4, 1))
rnn_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], 4, 1))

layers = [2, 1, 1]
batch_size = 128
num_epochs = 200
num_samples = rnn_train_x.shape[0]

scaled_layers = []
for layer in layers:
    scaled_layers.append(max(int(n_features * layer), 1))

early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# csv_logger = CSVLogger('./logs/nn-training-log.log')
checkpointer = ModelCheckpoint(
filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5, verbose=1)
weight_decay = 0.005 * batch_size / num_samples / num_epochs **0.5

print(scaled_layers)
print(batch_size)
print(num_epochs)
print(num_samples)
print(weight_decay)

# define model
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=4, activation='relu', 
                 input_shape=(n_timesteps,n_features)
                ))
#                ,   kernel_regularizer=L1L2(0.0001, 0.0001)))
model.add(Conv1D(filters=128, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(n_outputs))

# for layer_size in scaled_layers[1:-1]:
model.add(LSTM(scaled_layers[1], activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(scaled_layers[2], activation='relu')))
model.add(TimeDistributed(Dense(1)))

model.compile(loss='mae', optimizer=Adam(lr=0.1))
# fit network
model.fit(rnn_train_x, 
          rnn_train_y,
          validation_data = (rnn_test_x, rnn_test_y),
          epochs=num_epochs, 
          batch_size=batch_size, 
          callbacks=[early_stopping, checkpointer, reduce_lr],
          verbose=1)

model.load_weights('./weights/nn-weights.hdf5')
y_test_predictions = model.predict(rnn_test_x)

model_eval = evaluate_results(rnn_test_symbols, y_test_predictions, rnn_test_y)
model_eval.to_csv('./results/cnn-lstm-%d-%d-%d-epochs.csv' %(n_input, n_out, num_epochs))

print('Done')
model_eval

# 16-4, 200 epochs: 

## ConvLSTM

In [ ]:
## Time Distributed LSTM

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from AdamW import AdamW

# n_input = n_length * n_steps
n_length = 4
n_steps = 4
        
conv_train_x = rnn_train_x.reshape((rnn_train_x.shape[0], n_steps, 1, n_length, n_features))
conv_test_x = rnn_test_x.reshape((rnn_test_x.shape[0], n_steps, 1, n_length, n_features))

conv_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], 4, 1))
conv_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], 4, 1))

layers = [2, 1, 1]
batch_size = 128
num_epochs = 200
num_samples = rnn_train_x.shape[0]

scaled_layers = []
for layer in layers:
    scaled_layers.append(max(int(n_features * layer), 1))

early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# csv_logger = CSVLogger('./logs/nn-training-log.log')
checkpointer = ModelCheckpoint(
filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5, verbose=1)
weight_decay = 0.005 * batch_size / num_samples / num_epochs **0.5

print(scaled_layers)
print(batch_size)
print(num_epochs)
print(num_samples)
print(weight_decay)

# define model
model = Sequential()
model.add(ConvLSTM2D(256, (1,4), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
model.add(Flatten())
model.add(RepeatVector(n_outputs))
    
# for layer_size in scaled_layers[1:-1]:
model.add(GRU(scaled_layers[0], activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(scaled_layers[1], activation='relu')))
model.add(TimeDistributed(Dense(1)))

model.compile(loss='mae', optimizer=optimizers.RMSprop())
# fit network
model.fit(conv_train_x, 
          conv_train_y,
          validation_data = (conv_test_x, conv_test_y),
          epochs=num_epochs, 
          batch_size=batch_size, 
          callbacks=[early_stopping, checkpointer, reduce_lr],
          verbose=1)

model.load_weights('./weights/nn-weights.hdf5')
y_test_predictions = model.predict(conv_test_x)

model_eval = evaluate_results(rnn_test_symbols, y_test_predictions, rnn_test_y)
model_eval.to_csv('./results/convlstm2d-%d-%d-%d-epochs.csv' %(n_input, n_out, num_epochs))

print('Done')
model_eval

# 16-4, 200 epochs: 

## DeepTimeSeries

In [ ]:
# %load_ext autoreload
%autoreload 2

In [ ]:
from DeepTimeSeries.models.RNN2Dense import * 
from DeepTimeSeries.models.Seq2Seq import *
from keras import optimizers # RMSProp, Adagrad, Adadelta, Adamax, Nadam, Adam
from keras.regularizers import L1L2
from AdamW import AdamW

# hyperparameters =======================
n_memory_steps = 16 # time steps for encoder
n_forcast_steps = 8 # time steps for decoder
batch_size = 128 # batch size for training
epochs = 500 # epochs for training
test_model = 'Seq2Seq_2'    # 'RNN2Dense_1' / 'Seq2Seq_1' / 'Seq2Seq_2'
cell = 'GRU'           # 'SimpleRNN' / 'LSTM' / 'GRU'
# =======================================

rnn_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], 4, 1))
rnn_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], 4, 1))

print(rnn_train_x.shape)
print(rnn_train_y.shape)

# build model
if test_model == 'RNN2Dense_1':
    dts = RNN2Dense_1(rnn_train_x.shape[1:], rnn_train_y.shape[1:], cell, 300, (128,3))
elif test_model == 'RNN2Dense_2':
    dts = RNN2Dense_2(rnn_train_x.shape[1:], rnn_train_y.shape[1:], cell, 300, (128,3))
elif test_model == 'Seq2Seq_1':
    dts = Seq2Seq_1(rnn_train_x.shape[1:], rnn_train_y.shape[1:], cell, 300)
elif test_model == 'Seq2Seq_2':
    dts = Seq2Seq_2(rnn_train_x.shape[1:], rnn_train_y.shape[1:], cell, 500, False, None, 0., 0.) # L1L2(0.001, 0.001)
print(dts.summary())
# compile model
dts.compile(optimizer=optimizers.Nadam(), loss='mape', metrics='mae')

# train model
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
checkpointer = ModelCheckpoint(
filepath='./weights/nn-weights.hdf5', verbose=0, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-5, verbose=1)

dts.fit(rnn_train_x, 
              rnn_train_y, 
              validation_data=(rnn_test_x, rnn_test_y),
              batch_size=batch_size,   
              epochs=epochs, 
              callbacks=[early_stopping, checkpointer, reduce_lr],
              verbose=1)

y_test_predictions = dts.predict(rnn_test_x)

model_eval = evaluate_results(rnn_test_symbols, y_test_predictions, rnn_test_y)
model_eval.to_csv('./results/%s-%s-%d-%d-%d-epochs.csv' %(test_model, cell, n_input, n_out, num_epochs))

print('Done')
model_eval

# Seq2Seq1 LSTM: loss: 1.6082 - val_loss: 1.3650
# Seq2Seq1 GRU: loss: 0.7289 - val_loss: 1.1589
# Seq2Seq1 SimpeRNN: loss: 0.7611 - val_loss: 1.1223
# Seq2Seq2 LSTM: loss: 0.9287 - val_loss: 1.0096
# Seq2Seq2 GRU: loss: 0.5922 - val_loss: 1.3541 **
# Seq2Seq2 GRU L1L2: loss: 0.8083 - val_loss: 1.3500
# Seq2Seq2 GRU RMSProp: loss: 0.3581 - val_loss: 1.4363
# Seq2Seq2 GRU RMSProp L1L2 0.001: loss: 0.8680 - val_loss: 1.3121
# Seq2Seq2 GRU Adagrad: loss: 1.6082 - val_loss: 1.3650
# Seq2Seq2 GRU Adadelta: loss: 0.9741 - val_loss: 1.0678
# Seq2Seq2 GRU Adamax: loss: 0.6380 - val_loss: 1.3190
# Seq2Seq2 GRU Nadam: loss: 0.6843 - val_loss: 1.3261
# Seq2Seq2 SimpeRNN: loss: 0.7180 - val_loss: 1.2733
# RNN2Dense_1 LSTM: loss: 1.0986 - val_loss: 0.9767
# RNN2Dense_1 GRU: loss: 1.1712 - val_loss: 1.1004
# RNN2Dense_1 SimpeRNN: loss: 1.2591 - val_loss: 1.0768

# Seq2Seq2 200 epochs GRU L1L2 0.001: mae: 5.704368591308594 mape: 72.01702147722244 medae: 2.7182743549346924
# Seq2Seq2 500 epochs GRU L1L2 0.001:

## Attention Model

In [ ]:
import numpy as np
import tensorflow as tf
import os

from tensorflow import keras

K = keras.backend

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import RepeatVector, Concatenate, Activation
from tensorflow.python.keras.layers import Reshape, Input, Dense, Dot, LSTM
from tensorflow.python.keras import regularizers

from tensorflow.python.keras.models import load_model as keras_load_model

import matplotlib.pyplot as plt
import pylab


# Sometimes helpful to implement own softmax activation function to
# better manage calculations along specific axes.
def softmax_activation(x):
    e = K.exp(x - K.max(x, axis=1, keepdims=True))
    s = K.sum(e, axis=1, keepdims=True)
    return e / s


class AttentionModel(object):

    def __init__(self, x, y,
                 layer_1_rnn_units,
                 attn_dense_nodes=0,
                 epochs=100,
                 batch_size=128,
                 shared_attention_layer=True,
                 chg_yield=False,
                 float_type='float32',
                 regularization=(0.00001, '00001'),
                 window=52,
                 predict=1):
        K.clear_session()
        tf.reset_default_graph()
        self.set_learning(True)

        # Scientific computing uses 'float64' but
        # machine learning works much faster with 'float32'.
        self.float_type = float_type
        K.set_floatx(self.float_type)

        # Capture inputs to instance variables.
        self.x = x
        self.y = y
        self.epochs = epochs
        self.batch_size = batch_size
        self.shared_attention_layer = shared_attention_layer

        self.layer_1_rnn_units = layer_1_rnn_units
        self.layer_2_rnn_units = self.layer_1_rnn_units
        self.attn_dense_nodes = attn_dense_nodes

        self.num_obs = self.x.shape[0]
        self.input_len = self.x.shape[1]
        self.input_dims = self.x.shape[2]
        self.num_outputs = self.y.shape[1]

        self.regularization = regularization[0]

        assert self.x.shape[0] == self.y.shape[0]

        # Set the directory structure.
        self.model_dir = f'models//window_{window}_predict_{predict}//'

        self.model_name = f'{"yield_changes" if chg_yield==True else "yield_levels"}//' \
                          f'model_{layer_1_rnn_units}_rnn_{attn_dense_nodes}_dense_attn_' \
                          f'{epochs}_epochs_' \
                          f'{batch_size}_batch_' \
                          f'{"shared_attention" if shared_attention_layer else""}_' \
                          f'{"change_yield" if chg_yield else "level_yield"}_' \
                          f'{regularization[1]}_reg'

        # Activation function for the attention mechanism dense layer(s).
        self.attn_dense_activation = 'selu'
        self.attn_dense_initializer = 'lecun_normal'
        
    def delete_model(self):
        try:
            os.remove(f'{self.model_dir}{self.model_name}.h5')
        except Exception as e:
            print(e)

    def load_model(self):
        try:
            self.model = keras_load_model(f'{self.model_dir}{self.model_name}.h5',
                                          custom_objects={'softmax_activation': softmax_activation})
        except Exception as e:
            print(e)
            return False
        return True

    def save_model(self):
        try:
            self.model.save(f'{self.model_dir}{self.model_name}.h5')
        except Exception as e:
            print(e)
            return False
        return True

    def set_learning(self, learning):
        if learning:
            self.is_learning_phase = 1
            K.set_learning_phase(self.is_learning_phase)
            tf.keras.backend.set_learning_phase(True)
        else:
            self.is_learning_phase = 0
            K.set_learning_phase(self.is_learning_phase)
            tf.keras.backend.set_learning_phase(False)

    # Method that constructs shared layers. A shared layer means its learned parameters
    # are the same no matter where the layer is used in the neural network.
    def make_shared_layers(self):
        if self.regularization > 0.:
            self.kernel_reg = regularizers.l2(self.regularization)
            self.bias_reg = regularizers.l2(self.regularization)
            self.recurrent_reg = regularizers.l2(self.regularization)
            self.recurrent_dropout = 0.1
        else:
            self.kernel_reg = self.bias_reg = self.recurrent_reg = None
            self.recurrent_dropout = 0.0

        if self.shared_attention_layer:
            # This is an optional intermediate dense layer in the attention network.
            # If it is not present, the attention mechanism goes straight from inputs to weights.
            if self.attn_dense_nodes > 0:
                self.attn_middle_dense_layer = Dense(self.attn_dense_nodes,
                                                     kernel_regularizer=self.kernel_reg,
                                                     bias_regularizer=self.bias_reg,
                                                     activation=self.attn_dense_activation,
                                                     kernel_initializer=self.attn_dense_initializer,
                                                     name='attention_mid_dense_shared')

            # This is the layer in the attention mechanism that gives the attention weights.
            self.attention_final_dense_layer = Dense(1,
                                                     kernel_regularizer=self.kernel_reg,
                                                     bias_regularizer=self.bias_reg,
                                                     activation=self.attn_dense_activation,
                                                     kernel_initializer=self.attn_dense_initializer,
                                                     name='attention_final_dense_shared')

        # Output-level LSTM cell.
        self.layer_2_LSTM_cell = LSTM(self.layer_2_rnn_units,
                                      kernel_regularizer=self.kernel_reg,
                                      recurrent_regularizer=self.recurrent_reg,
                                      bias_regularizer=self.bias_reg,
                                      recurrent_dropout=self.recurrent_dropout,
                                      return_state=True,
                                      name='layer_2_LSTM')

        # Final output (i.e., the prediction).
        self.dense_output = Dense(1,
                                  kernel_regularizer=self.kernel_reg,
                                  bias_regularizer=self.bias_reg,
                                  activation='linear',
                                  name='dense_output')

    # Builds the neural network. An LSTM+attention model doesn't need this much code.
    # This method is long because it sets lots of layer parameters and because
    # it handles four contingencies: (1) whether the attention mechanism is
    # always the same or is different for every prediction node, and (2) whether or
    # not the attention mechanism has an intermediate dense layer.
    def build_attention_rnn(self):
        self.make_shared_layers()

        inputs = Input(shape=(self.input_len, self.input_dims), dtype=self.float_type)

        X = LSTM(self.layer_1_rnn_units,
                 kernel_regularizer=self.kernel_reg,
                 recurrent_regularizer=self.recurrent_reg,
                 bias_regularizer=self.bias_reg,
                 recurrent_dropout=self.recurrent_dropout,
                 return_sequences=True)(inputs)

        X = Reshape((self.input_len, self.layer_2_rnn_units))(X)

        h_start = Input(shape=(self.layer_2_rnn_units,), name='h_start')
        c_start = Input(shape=(self.layer_2_rnn_units,), name='c_start')
        h_prev = h_start
        c_prev = c_start

        outputs = list()

        # This section constructs the attention mechanism and the output-level LSTM
        # layer that leads to the predictions.
        #
        # There is an extra LSTM cell that is not attached to any prediction but
        # which begins the output-level RNN sequence. This avoids sending in a bunch
        # of zero values to the first usage of the attention mechanism.
        #
        # One way to avoid this extra LSTM cell might be to set the LSTM intial state
        # tensors "h_start" and "c_start" as trainable (instead of zeros).
        for t in range(self.num_outputs + 1):
            h_prev_repeat = RepeatVector(self.input_len)(h_prev)
            joined = Concatenate(axis=-1)([X, h_prev_repeat])

            if self.attn_dense_nodes > 0:
                if self.shared_attention_layer:
                    joined = self.attn_middle_dense_layer(joined)
                else:
                    joined = Dense(self.attn_dense_nodes,
                                   kernel_regularizer=self.kernel_reg,
                                   bias_regularizer=self.bias_reg,
                                   activation=self.attn_dense_activation,
                                   kernel_initializer=self.attn_dense_initializer,
                                   name=f'attention_mid_dense_{t}')(joined)

            if self.shared_attention_layer:
                e_vals = self.attention_final_dense_layer(joined)
            else:
                e_vals = Dense(1,
                               kernel_regularizer=self.kernel_reg,
                               bias_regularizer=self.bias_reg,
                               activation=self.attn_dense_activation,
                               kernel_initializer=self.attn_dense_initializer,
                               name=f'attention_final_dense_{t}')(joined)

            alphas = Activation(softmax_activation, name=f'attention_softmax_{t}')(e_vals)
            attentions = Dot(axes=1)([alphas, X])

            h_prev, _, c_prev = self.layer_2_LSTM_cell(attentions, initial_state=[h_prev, c_prev])

            if t > 0:
                out = self.dense_output(h_prev)
                outputs.append(out)

        self.model = Model(inputs=[inputs, h_start, c_start], outputs=outputs)
        self.model.compile(loss='mse', optimizer='adam', metrics=['mse'])

        print(self.model.summary())

    def fit_model(self):
        self.set_learning(True)

        h_start = np.zeros((self.num_obs, self.layer_2_rnn_units))
        c_start = np.zeros((self.num_obs, self.layer_2_rnn_units))

        y_split = np.split(self.y, indices_or_sections=self.num_outputs, axis=1)

        self.model.fit([self.x, h_start, c_start],
                       y_split,
                       epochs=self.epochs,
                       batch_size=self.batch_size,
                       shuffle=True,
                       verbose=2,
                       validation_split=0.1)

    def calculate_attentions(self, x_data):
        self.set_learning(False)

        softmax_layer_names = [f'attention_softmax_{t}' for t in range(self.num_outputs + 1)]
        softmax_layers = list()

        for i, layer_name in enumerate(softmax_layer_names):
            if i == 0:
                continue
            intermediate_layer = Model(inputs=self.model.input,
                                       outputs=self.model.get_layer(layer_name).output)
            softmax_layers.append(intermediate_layer)

        num_obs = x_data.shape[0]
        attention_map = np.zeros((num_obs, self.num_outputs, self.input_len))

        h_start = np.zeros((1, self.layer_2_rnn_units))
        c_start = np.zeros((1, self.layer_2_rnn_units))

        for t in range(num_obs):
            print(t)
            for l_num, layer in enumerate(softmax_layers):
                softmax_results = layer.predict([np.expand_dims(x_data[t], axis=0),
                                                 h_start,
                                                 c_start])
                softmax_results = softmax_results[0, :, 0]
                attention_map[t, l_num, :] = softmax_results

        return attention_map

    def heatmap(self, data, title_supplement=None):
        plt.rcParams['axes.labelweight'] = 'bold'
        plt.rcParams['axes.labelsize'] = 22
        plt.rcParams['axes.titlesize'] = 22
        plt.rcParams['axes.titleweight'] = 'bold'
        plt.rcParams['xtick.labelsize'] = 18
        plt.rcParams['ytick.labelsize'] = 18
        plt.rcParams['axes.titlepad'] = 12
        plt.rcParams['axes.edgecolor'] = '#000000'  # '#FD5E0F'

        # Other common color schemes: 'viridis'  'plasma'  'gnuplot'
        color_map = 'inferno'
        pylab.pcolor(data, cmap=color_map, vmin=0.)
        pylab.colorbar()

        num_predictions = data.shape[0]
        num_timesteps = data.shape[1]

        if num_predictions == 4:
            pylab.yticks([0.5, 1.5, 2.5, 3.5], ['t+1', 't+2', 't+3', 't+4'])
            pylab.ylabel('y: t+1 to t+4')

            plt.axhline(y=1., xmin=0.0, xmax=51.0, linewidth=1, color='w')
            plt.axhline(y=2., xmin=0.0, xmax=51.0, linewidth=1, color='w')
            plt.axhline(y=3., xmin=0.0, xmax=51.0, linewidth=1, color='w')

        elif num_predictions == 1:
            pylab.yticks([0.5], ['t+1'])
            pylab.ylabel('y: t+1')

        assert num_timesteps == 52

        pylab.xticks([1.5, 11.5, 21.5, 31.5, 41.5, 51.5],
                     ['t-50', 't-40', 't-30', 't-20', 't-10', 't'])
        pylab.xlabel('x: t-51 to t')

        pylab.title(f'{self.model_name} {title_supplement}')

        mng = plt.get_current_fig_manager()
        mng.window.showMaximized()
        pylab.show()